# RETRIEVING DATA FROM TWITTER

In [1]:
# General:
import tweepy           # To consume Twitter's API
import pandas as pd     # To handle data
import numpy as np      # For number computing

# For plotting and visualization:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import csv
import mysql.connector

from nltk import bigrams
import networkx as nx

from sqlalchemy import create_engine
import pymysql

In [ ]:
# Consume:
CONSUMER_KEY    = 'iaW7mNbmXJNzK7H0lYp9w9lvl'
CONSUMER_SECRET = '39od1VNgE1t8RKzddZjSUFMm9rcGBPcR5DSSjPAWNf5c7EE0K2'

# Access:
ACCESS_TOKEN  = '2933481336-lWtyqSrObN7fQFgN9yU6XDmiV4WOlayNoes741Z'
ACCESS_SECRET = 'xgpKoz1Piphhu3DGznZjS5WBGLITyJ0IwKX6U1vVzWSHW'

In [ ]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [ ]:
search_words = "Xbox"
new_search=search_words + "-filter:retweets"

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
##### THIS METHOD IS NOT USED###
tweets = tweepy.Cursor(api.search, 
                           q=search_words,
                           lang="en").items(2000)

with open('PS5_tweets','w',encoding='utf-8',newline='') as file:
    writer=csv.writer(file)
    writer.writerow(["tweet","id", "date","likes","RTs","source","len","followers","friends","Name_user","Statuses_count","location","verified"])
    for tweet in tweets:
        writer.writerow([tweet.text,tweet.id,tweet.created_at,tweet.favorite_count,tweet.retweet_count,tweet.source, len(tweet.text),tweet.user.followers_count,tweet.user.friends_count,tweet.user.screen_name, tweet.user.statuses_count,tweet.user.location,tweet.user.verified])

In [ ]:
tweets = tweepy.Cursor(api.search, 
                           q=search_words,
                           lang="en").items(2000)
users_locs = [[tweet.text,tweet.id,tweet.created_at,tweet.favorite_count,tweet.retweet_count,tweet.source, len(tweet.text),tweet.user.followers_count,tweet.user.friends_count,tweet.user.screen_name, tweet.user.statuses_count,tweet.user.location,tweet.user.verified] for tweet in tweets]

In [ ]:
tweet_text = pd.DataFrame(data=users_locs, columns=["tweet","id", "date","likes","RTs","source","len","followers","friends","Name_user","Statuses_count","location","verified"])
display(tweet_text)

#  CLEANING THE DATA AND ADDING NEW COLUMN

In [3]:
### SAVING DF TO CSV FILE
tweet_text.to_csv("Xbox_series_x.csv",index=False)

NameError: name 'tweet_text' is not defined

In [4]:
xbox = pd.read_csv (r'C:\Users\keven\Desktop\Xbox_series_x.csv')
print(len(xbox))


2000


In [5]:
xbox.rename(columns={'date': 'Date','likes':'Likes','source':'Source','len':'Length','followers':'Followers','friends':'Friends','Name_user':'Name_User','Statuses_count':'Statuses_Count','verified':'Verified'}, inplace=True)

In [6]:
xbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
tweet             2000 non-null object
id                2000 non-null int64
Date              2000 non-null object
Likes             2000 non-null int64
RTs               2000 non-null int64
Source            1999 non-null object
Length            2000 non-null int64
Followers         2000 non-null int64
Friends           2000 non-null int64
Name_User         2000 non-null object
Statuses_Count    2000 non-null int64
location          2000 non-null object
Verified          2000 non-null bool
Tweets_Cleaned    2000 non-null object
Subjectivity      2000 non-null float64
Polarity          2000 non-null float64
Sentiment         2000 non-null object
Hashtag           2000 non-null int64
dtypes: bool(1), float64(2), int64(8), object(7)
memory usage: 267.7+ KB


In [7]:
### DROPPIN ALL NAs AND DUPLICATES
xbox.drop_duplicates(keep=False,inplace=True) 


In [8]:
# No duplicate row
print(len(xbox))

2000


In [9]:
#### location have a lot of missing values. lets take a deeper look####
xbox.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 18 columns):
tweet             2000 non-null object
id                2000 non-null int64
Date              2000 non-null object
Likes             2000 non-null int64
RTs               2000 non-null int64
Source            1999 non-null object
Length            2000 non-null int64
Followers         2000 non-null int64
Friends           2000 non-null int64
Name_User         2000 non-null object
Statuses_Count    2000 non-null int64
location          2000 non-null object
Verified          2000 non-null bool
Tweets_Cleaned    2000 non-null object
Subjectivity      2000 non-null float64
Polarity          2000 non-null float64
Sentiment         2000 non-null object
Hashtag           2000 non-null int64
dtypes: bool(1), float64(2), int64(8), object(7)
memory usage: 283.2+ KB


In [10]:
### these a very messy data. not everyone shares his location. With that said, we will be removing this column
xbox.location.unique()

array(['I Eat Animals Everyday...', 'UK', 'Unknown', 'Ur moms house',
       'Los Angeles, CA', 'he/him', 'Hell 😈', 'England',
       'London, United Kingdom', 'Australia', 'Bordeaux France',
       'Louisiana, USA', 'Alcorcón, España', 'Wolverhampton, England',
       'Deutschland', 'world wide web', 'United Kingdom',
       'Rockford, IL USA', 'Everton', 'Gamertag Enter Skies',
       'Valladolid', 'Westham, England', 'Hong Kong', 'out of this world',
       'Mareven ✧•°:', 'Waltham, MA', 'Bratfud', 'Michigan, USA',
       'Purgatory, Florida', 'United States', 'US', 'Las Vegas', 'India',
       'arizona', 'Kettering', 'Lancashire', 'USA ', 'Mooloolaba',
       'California, USA', 'Paris, France', 'Italia',
       'West Midlands, England', 'Minnesota, USA',
       'East Midlands, England', 'New Zealand', 'a house in the uk',
       'Worldwide', 'London', 'PSN: Donny Ray', 'Louisville, KY',
       'Probably Hell', 'USA', 'United World', 'Hamburg Altona',
       'Detroit, Michigan', 'Ep

In [11]:
xbox.drop(['location'], axis=1, inplace=True)

In [12]:
xbox.dropna(inplace=True)
print(xbox.head())

                                               tweet                   id  \
0  @PyvosGaming @NotDollie They never mentioned p...  1337322115398135808   
1  Better than Black Friday! 🧐\n\nSomehow, mere w...  1337322016047828995   
2  @PS5_Restockk Thanks to him I got yesterday a ...  1337321958292197378   
3  @ASTROGaming I don’t think ur a50 product work...  1337321891057389569   
4  Win An #Xbox Series X With #Android Headlines ...  1337321636492677122   

                  Date  Likes  RTs               Source  Length  Followers  \
0  2020-12-11 09:03:33      0    0      Twitter Web App     140        166   
1  2020-12-11 09:03:09      0    0      Twitter Web App     140       5885   
2  2020-12-11 09:02:55      0    0  Twitter for Android      93          1   
3  2020-12-11 09:02:39      0    0   Twitter for iPhone     107         80   
4  2020-12-11 09:01:39      0    1      Twitter Web App     108     485544   

   Friends        Name_User  Statuses_Count  Verified  \
0      159 

In [13]:
xbox.index = range(len(xbox))
xbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 17 columns):
tweet             1999 non-null object
id                1999 non-null int64
Date              1999 non-null object
Likes             1999 non-null int64
RTs               1999 non-null int64
Source            1999 non-null object
Length            1999 non-null int64
Followers         1999 non-null int64
Friends           1999 non-null int64
Name_User         1999 non-null object
Statuses_Count    1999 non-null int64
Verified          1999 non-null bool
Tweets_Cleaned    1999 non-null object
Subjectivity      1999 non-null float64
Polarity          1999 non-null float64
Sentiment         1999 non-null object
Hashtag           1999 non-null int64
dtypes: bool(1), float64(2), int64(8), object(6)
memory usage: 252.0+ KB


In [14]:
### CLEANING TWEETS FUNCTION ####
import re
def cleanTxt(text):
    text= re.sub(r'@[A-Za-z0-9\_]+','',text) #Removed @mentions
    text= re.sub(r'#','',text)#Remove the '#' symbol
    text= re.sub(r'RT[\s]+','',text)#Remove retweets
    text= re.sub(r'https?:\/\/\S+','',text) #Remove links
    text= re.sub(r'ps5','',text)
    return text

In [15]:
#### CREATING A NEW COLUMN WHERE THE CLEANED TWEETS ARE STORED ###
xbox['Tweets_Cleaned'] = xbox['tweet'].apply(cleanTxt)

In [16]:
### Remove emojis 
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)


In [17]:
### CREATING A LIST THAT CONTAINS ALL THE REMOVED EMOJIS
list2=[]
for i in xbox.Tweets_Cleaned:
    list2.append(remove_emojis(i))

In [18]:
### CLEANING THE TWEETS FROM EMOJIS
for i in range(len(xbox)):
    xbox.loc[[i],['Tweets_Cleaned']]= list2[i]

In [19]:
### Cleaniing the unclead tweet
xbox.drop(['tweet'], axis=1, inplace=True)

In [20]:
### unique###
xbox.Source.unique()

array(['Twitter Web App', 'Twitter for Android', 'Twitter for iPhone',
       'Twitter Media Studio', 'Twitter for Advertisers (legacy)',
       'IFTTT', 'Clickclickme', 'dlvr.it', 'WordPress.com',
       'Racing Snail Post Service', 'Streamlabs Twitter',
       'Game-eXperience.it', 'Twibble.io', 'Nlife CMS', 'XboxTrackr',
       'Crowdfire App', 'TweetDeck', 'PostPickr', 'Hootsuite Inc.',
       'Twitter for iPad', 'SocialFlow', 'GamingBolt Application',
       'LinkedIn', 'Twitter for Mac', 'The Video Games',
       'Xbox One Social', 'NewsTechBot', 'Tweetbot for iΟS',
       'feather free for iOS ', 'Echobox', 'Buffer',
       'App for FintechZoom.com', 'Skyepod.id', 'SocialChamp IO ',
       'MeetEdgar', 'ViralSweep App', 'SocialRabbit Plugin', 'Publer ',
       'Paper.li', 'Latesttoy.com', 'Periscope', 'XBOR PM',
       'Derek T McKinney', 'DataScienceInfo', 'Twitterrific for iOS',
       'Instagram', 'Upflow', 'Chorus publishing platform',
       'NeoGAF Threads', 'Microsoft Azu

In [21]:
# Replace the most important ones, we care about these
xbox.replace(to_replace ="Twitter for iPhone", 
                 value ="ios", inplace=True) 
xbox.replace(to_replace ="Twitter for Android", 
                 value ="Android", inplace=True) 
xbox.replace(to_replace ="Twitter Web App", 
                 value ="Web App", inplace=True) 
xbox.replace(to_replace ="Twitter for iPad", 
                 value ="ios", inplace=True)

In [22]:
# Replacing not important ones by others
list3=['ios','Android','Web App']

for i in xbox.Source:
    if i not in list3:
        xbox.replace(to_replace=i, value='Others',inplace=True)

In [23]:
### Unique ids ####
len(xbox.Name_User.unique())

1668

In [24]:
#### some tweeted more than once###
len(xbox.id.unique())

1999

In [25]:
#Get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Get Polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [26]:
#Create two new columns
xbox['Subjectivity']=xbox['Tweets_Cleaned'].apply(getSubjectivity)
xbox['Polarity']=xbox['Tweets_Cleaned'].apply(getPolarity)

In [27]:
def getAnalysis(score):
    if score <0:
        return 'Negative'
    elif score==0:
        return 'Neutral'
    else:
        return 'Positive'
xbox['Sentiment']=xbox['Polarity'].apply(getAnalysis)

In [28]:
xbox.Verified.unique()

array([False,  True])

In [29]:
xbox.loc[xbox.Verified == True, 'Verified'] = 1
xbox.loc[xbox.Verified == False, 'Verified'] = 0

In [30]:
xbox.Verified.unique()

array([0, 1], dtype=int64)

# Seperating the two dataframes

In [32]:
#drop Column that are not for users
Users=xbox.drop(['id','Tweets_Cleaned','Date','RTs','Source','Length','Subjectivity','Polarity','Sentiment',"Likes"], axis=1)

In [33]:
#Drop duplicate name_User
Users.drop_duplicates(subset=['Name_User'],inplace=True)

In [34]:
#Add User id
Users["User_id"]=range(1, len(Users)+1)
Users.index = range(len(Users))

In [35]:
xbox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 16 columns):
id                1999 non-null int64
Date              1999 non-null object
Likes             1999 non-null int64
RTs               1999 non-null int64
Source            1999 non-null object
Length            1999 non-null int64
Followers         1999 non-null int64
Friends           1999 non-null int64
Name_User         1999 non-null object
Statuses_Count    1999 non-null int64
Verified          1999 non-null int64
Tweets_Cleaned    1999 non-null object
Subjectivity      1999 non-null float64
Polarity          1999 non-null float64
Sentiment         1999 non-null object
Hashtag           1999 non-null int64
dtypes: float64(2), int64(9), object(5)
memory usage: 250.0+ KB


In [36]:
#Drop colums that are useful for User and the tweet datafram
xbox_tweets=xbox.drop(['Followers','Friends','Verified','Statuses_Count'],axis=1)

In [37]:
#Create Dictionary that has the name_user as key and user_id as value
Dict1=pd.Series(Users.User_id.values,index=Users.Name_User).to_dict()
print(Dict1)

{'BillyHoWCR': 1, 'consoledeals': 2, 'LizMarreroPR': 3, 'that_hanes_guy': 4, 'Androidheadline': 5, 'GunBangJosh': 6, 'imbbprang222': 7, 'Samthemanmanmen': 8, 'Gfinity': 9, 'BluemouthInt': 10, 'AsoboStudio': 11, 'ViceVersaLiving': 12, 'flipper83': 13, 'SuperFluxy': 14, 'Maskulinbaby': 15, 'dorothyrash36': 16, 'MadeaGoestoNam': 17, 'Gamblerfeeds': 18, 'GillyTheGamer': 19, 'StockNotifierUk': 20, 'GreenToneEnviro': 21, 'kkashif23': 22, 'TOFnews': 23, 'FlightSimNewss': 24, 'Enea55198807': 25, 'bitcoinconnect': 26, 'AyeSkies': 27, 'juarezN7': 28, 'BabyBoyLuni': 29, 'James50023680': 30, 'westhamnews2019': 31, 'JahangeerDM': 32, 'Cold_Fudge': 33, 'flowerkinghere': 34, 'DaveWShanahan': 35, 'Siorsid': 36, 'bambooney': 37, 'JustinN61622099': 38, 'marriedgamer711': 39, 'Wo1fB1ood': 40, 'Assassin4than': 41, 'XPLootGaming': 42, 'thur5day': 43, 'YeetLeet5': 44, 'amyshulk': 45, 'consolemine': 46, 'ShawnRevan18': 47, 'onmsft': 48, 'dosyboy20000': 49, 'Xboxthusiast': 50, 'ak4shr4j_': 51, 'Uberutang': 52

In [38]:
#Creat a list containing every id mapped to the user
list1=[]
for i in range(len(xbox_tweets)):
    list1.append(Dict1.get(xbox_tweets['Name_User'].values[i]))

In [39]:
#Addi the row
xbox_tweets["User_id"]=list1

In [40]:
display(xbox_tweets)

,id,Date,Likes,RTs,Source,Length,Name_User,Tweets_Cleaned,Subjectivity,Polarity,Sentiment,Hashtag,User_id
0,1337322115398135808,2020-12-11 09:03:33,0,0,Web App,140,BillyHoWCR,They never mentioned platform exclusive. The...,0.000000,0.000000,Neutral,0,1
1,1337322016047828995,2020-12-11 09:03:09,0,0,Web App,140,consoledeals,"Better than Black Friday! \n\nSomehow, mere we...",0.477778,-0.069444,Negative,0,2
2,1337321958292197378,2020-12-11 09:02:55,0,0,Android,93,LizMarreroPR,Thanks to him I got yesterday a Xbox series x...,0.200000,0.200000,Positive,0,3
3,1337321891057389569,2020-12-11 09:02:39,0,0,ios,107,that_hanes_guy,I don’t think ur a50 product works right on m...,0.580087,0.407359,Positive,0,4
4,1337321636492677122,2020-12-11 09:01:39,0,1,Web App,108,Androidheadline,Win An Xbox Series X With Android Headlines - ...,0.400000,0.800000,Positive,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,1337189682472513536,2020-12-11 00:17:18,2,0,ios,140,freakytj,"Damn, I’m sorry man. Kept thinking you got...",1.000000,-0.500000,Negative,0,1665
1995,1337189608556294144,2020-12-11 00:17:01,0,0,Android,72,MeepzorMeehee,Yeah looks like I'll be on the lookout for Xb...,0.000000,0.000000,Neutral,0,1666
1996,1337189562683166721,2020-12-11 00:16:50,0,0,Others,141,bitcoinconnect,PlayStation 5 and Xbox Series X and S sold out...,0.388889,0.166667,Positive,0,26
1997,1337189546157543424,2020-12-11 00:16:46,3,0,Web App,53,shamshandwich,Perfect Dark 2 (from Xbox 360)\n\nNow for Xbox...,0.700000,0.425000,Positive,0,1667


In [41]:
xbox_tweets.drop('Name_User', axis=1, inplace=True)
display(xbox_tweets)

,id,Date,Likes,RTs,Source,Length,Tweets_Cleaned,Subjectivity,Polarity,Sentiment,Hashtag,User_id
0,1337322115398135808,2020-12-11 09:03:33,0,0,Web App,140,They never mentioned platform exclusive. The...,0.000000,0.000000,Neutral,0,1
1,1337322016047828995,2020-12-11 09:03:09,0,0,Web App,140,"Better than Black Friday! \n\nSomehow, mere we...",0.477778,-0.069444,Negative,0,2
2,1337321958292197378,2020-12-11 09:02:55,0,0,Android,93,Thanks to him I got yesterday a Xbox series x...,0.200000,0.200000,Positive,0,3
3,1337321891057389569,2020-12-11 09:02:39,0,0,ios,107,I don’t think ur a50 product works right on m...,0.580087,0.407359,Positive,0,4
4,1337321636492677122,2020-12-11 09:01:39,0,1,Web App,108,Win An Xbox Series X With Android Headlines - ...,0.400000,0.800000,Positive,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1994,1337189682472513536,2020-12-11 00:17:18,2,0,ios,140,"Damn, I’m sorry man. Kept thinking you got...",1.000000,-0.500000,Negative,0,1665
1995,1337189608556294144,2020-12-11 00:17:01,0,0,Android,72,Yeah looks like I'll be on the lookout for Xb...,0.000000,0.000000,Neutral,0,1666
1996,1337189562683166721,2020-12-11 00:16:50,0,0,Others,141,PlayStation 5 and Xbox Series X and S sold out...,0.388889,0.166667,Positive,0,26
1997,1337189546157543424,2020-12-11 00:16:46,3,0,Web App,53,Perfect Dark 2 (from Xbox 360)\n\nNow for Xbox...,0.700000,0.425000,Positive,0,1667


# Lets try to fit the data in

In [42]:
##### Creating a database #####
mydb=mysql.connector.connect(
    host="localhost",
    user='root',
    password="password")

mycursor = mydb.cursor()



In [44]:
mycursor.execute("CREATE DATABASE Project5")

In [45]:
mydb=mysql.connector.connect(
    host="localhost",
    user='root',
    password="password",
    database="Project5")

In [46]:
my_cursor = mydb.cursor()

In [47]:
Users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1668 entries, 0 to 1667
Data columns (total 7 columns):
Followers         1668 non-null int64
Friends           1668 non-null int64
Name_User         1668 non-null object
Statuses_Count    1668 non-null int64
Verified          1668 non-null int64
Hashtag           1668 non-null int64
User_id           1668 non-null int32
dtypes: int32(1), int64(5), object(1)
memory usage: 84.8+ KB


In [59]:
my_cursor = mydb.cursor()

In [60]:
my_cursor.execute("CREATE TABLE xbox_users1 (User_id INT ,Followers INT, Friends INT, Name_User TEXT, Verified BOOLEAN,Statuses_Count INT, Primary key(User_id))")

In [52]:
def insertVariblesIntoTable(User_id2,  Followers, Friends, Name_User, Verified,Statuses_Count):
    try:
        connection = mysql.connector.connect(host="localhost",
                                                    user='root',
                                                    password="password",
                                                    database="Project5")
        cursor = connection.cursor()
        mySql_insert_query = """INSERT INTO xbox_users (User_id2,  Followers, Friends, Name_User, Verified,Statuses_Count) 
                                VALUES (%s, %s, %s, %s, %s,%s) """

        recordTuple = (User_id2,  Followers, Friends, Name_User, Verified,Statuses_Count)
        cursor.execute(mySql_insert_query, recordTuple)
        connection.commit()
        print("Record inserted successfully into xbox_users table")

    except mysql.connector.Error as error:
        print("Failed to insert into MySQL table {}".format(error))

    finally:
        if (connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [53]:
for i in range(len(Users)):
    insertVariblesIntoTable(int(Users.loc[i,"User_id"]),int(Users.loc[i,"Followers"]), int(Users.loc[i,"Friends"]),Users.loc[i,"Name_User"],int(Users.loc[i,"Verified"]),int(Users.loc[i,"Statuses_Count"]))

Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connect

MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbo

Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connect

Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connect

Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connect

...

Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connect

Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connect

Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connect

Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connect

Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connection is closed
Record inserted successfully into xbox_users table
MySQL connect

In [61]:
my_cursor = mydb.cursor()

In [63]:
my_cursor.execute("CREATE TABLE xbox_tweets (id2 BIGINT ,user_id2 INT,Date DATETIME,Likes INT,RTs INT, Source TEXT, length INT, Tweets_Cleaned TEXT, Subjectivity FLOAT, Polarity FLOAT, Sentiment TEXT, Primary key(id2),FOREIGN KEY (User_id2) REFERENCES Project5.xbox_users(User_id2))")

In [68]:
def insertVariblesIntoTable(id2 ,user_id2,Date,Likes,RTs, Source, length, Tweets_Cleaned, Subjectivity, Polarity, Sentiment):
    try:
        connection = mysql.connector.connect(host="localhost",
                                                    user='root',
                                                    password="password",
                                                    database="Project5")
        cursor = connection.cursor()
        mySql_insert_query = """INSERT INTO xbox_tweets (id2 ,user_id2,Date,Likes,RTs, Source, length, Tweets_Cleaned, Subjectivity, Polarity, Sentiment) 
                                VALUES (%s, %s, %s, %s, %s, %s,%s,%s,%s,%s,%s) """

        recordTuple = (id2 ,user_id2,Date, Likes,RTs, Source, length, Tweets_Cleaned, Subjectivity, Polarity, Sentiment)
        cursor.execute(mySql_insert_query, recordTuple)
        connection.commit()
        print("Record inserted successfully into ps5users3 table")

    except mysql.connector.Error as error:
        print("Failed to insert into MySQL table {}".format(error))

    finally:
        if (connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [69]:
for i in range(len(xbox_tweets)):
    insertVariblesIntoTable(int(xbox_tweets.loc[i,"id"]),int(xbox_tweets.loc[i,"User_id"]),xbox_tweets.loc[i,"Date"], int(xbox_tweets.loc[i,"Likes"]), int(xbox_tweets.loc[i,"RTs"]),xbox_tweets.loc[i,"Source"], int(xbox_tweets.loc[i,"Length"]),xbox_tweets.loc[i,"Tweets_Cleaned"],xbox_tweets.loc[i,"Subjectivity"],xbox_tweets.loc[i,"Polarity"], xbox_tweets.loc[i,"Sentiment"])

Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed

Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed

Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed

Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed

MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table

...

Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed

Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed

Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed

Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed

Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed
Record inserted successfully into ps5users3 table
MySQL connection is closed

# SQL queries are found into the first python code